In [ ]:
!python --version

Python 3.10.12


# Install packages

In [1]:
! pip install streamlit -q
! pip install openai -q
! pip install langchain -q
! pip install pandas -q
! pip install mysql-connector-python -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0

# Creat the "endpoint IP" needed for webpage, copy the shown IP

In [2]:
!wget -q -O - ipv4.icanhazip.com

34.139.77.21


# Create the streamlit app file

In [3]:
%%writefile app.py

import streamlit as st
import random
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import StreamlitChatMessageHistory
from langchain import OpenAI, ConversationChain
import mysql.connector
from mysql.connector import Error
import pandas as pd
from pandas import DataFrame, read_csv, read_sql_query

st.set_page_config(layout="wide")
if 'caption' not in st.session_state:
    st.session_state.caption = []
def create_connection():
    try:
        connection = mysql.connector.connect(host='dbnewyorkcartoon.cgyqzvdc98df.us-east-2.rds.amazonaws.com',
                                             database='new_york_cartoon',
                                             user='dbuser',
                                             password='Sql123456')
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            return connection
    except Error as e:
        print("Error while connecting to MySQL", e)
        return None

def select(q):
    connection = create_connection()
    if connection:
        try:
            df = read_sql_query(q, connection)
            return df
        except Error as e:
            print("Error executing the query:", e)
        finally:
            connection.close()
    return None

col1, col2 = st.columns(2)
with col1:
    form = st.form("my_form")
    GPT_API = form.text_input('Your GPT API:', key="API", type="password")
    form.form_submit_button("Submit")
    option_function = st.selectbox('Choose a function', ('Inspiration', 'Get Help from GPT', 'Funniness prediction','topic model graph'))

with col2:
    if option_function =='Inspiration' and st.button(':red[Give me some inspiration]'):
        random_img = random.randint(509, 853)
        random_integers = [random.randint(0, 19) for _ in range(3)]
        inspiration_URL = select("""SELECT image_url FROM base""")
        inspiration_caption = select(f"""SELECT caption FROM result WHERE contest_num={random_img}""")
        st.image(inspiration_URL.iloc[random_img+1, 0])
        st.write(inspiration_caption.iloc[random_integers[0],0])
        st.write(inspiration_caption.iloc[random_integers[1],0])
        st.write(inspiration_caption.iloc[random_integers[2],0])

    if option_function =='Get Help from GPT':
        if not st.session_state.API:
            st.title(':red[You need to enter the API!]')
        else:
            msgs = StreamlitChatMessageHistory(key="langchain_messages")
            memory = ConversationBufferMemory(chat_memory=msgs)
            template = """You are an AI chatbot having a conversation with a human.

            {history}
            Human: {human_input}
            AI: """
            prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)
            prompt_remember = PromptTemplate(
                input_variables=["description"],
                template="You are assisting someone trying to think of funny captions for a New York cartoon. Please remember this scene as the description of a cartoon: {description}")
            prompt_caption = PromptTemplate(
                input_variables=["caption"],
                template="Please remember this caption fot the cartoon: {caption}")
            prompt_suggestion = PromptTemplate(
                input_variables=["suggestion"],
                template="The user wants some more advice. One way of making a cartoon funnier is to {suggestion} With that in mind, please talk directly to the user to suggest ways the user can adapt their ideas to create a funnier caption. Respond in no more than three sentences.")
            llm_chain = LLMChain(llm=OpenAI(openai_api_key=st.session_state.API), prompt=prompt, memory=memory)

            with st.form(key='my_form2'):
                description_location = st.text_area('Please describe the content of the cartoon in as much detail as possible:', height = 2)
                submit_button_descriotion = st.form_submit_button(label='Submit the description of the cartoon')
            with st.form(key='my_form3'):
                caption_1 = st.text_input('Write your caption:')
                submit_button_caption = st.form_submit_button(label='Submit the caption')
            with st.form(key="my_form4"):
                option_help = st.selectbox('Choose a kind of help to ask GPT', ('use more specific language', 'make reference to all important elements of the image', 'use langauge with more than one interpretation','avoid making the joke too obvious'))
                help_button = st.form_submit_button(label='Get help')
            if st.button('Reset chat'):
                del st.session_state.langchain_messages
                msgs = StreamlitChatMessageHistory(key="langchain_messages")

            if submit_button_caption:
                st.session_state.caption.append(caption_1)
                prompt = prompt_caption.format(caption=caption_1)
                response = llm_chain.run(prompt)
                st.chat_message("ai").write(response)
            if submit_button_descriotion:
                prompt = prompt_remember.format(description=description_location)
                response = llm_chain.run(prompt)
                st.chat_message("ai").write(response)
            if help_button:
                prompt = prompt_suggestion.format(suggestion=option_help)
                response = llm_chain.run(prompt)
                st.chat_message("ai").write(response)

    if option_function =='Funniness prediction':
        st.title('Wait for further development!')

    if option_function =='topic model graph':
        st.title('Wait for further development!')

if option_function =='Get Help from GPT':
  with col1:
    with st.expander("View your submitted captions:"):
      if st.button("clear caption history"):
        st.session_state.caption = []
      st.write(st.session_state.caption)
  with col2:
    with st.expander("View suggestion history:"):
      count = int(len(msgs.messages)/2)
      for i in range(count):
          st.chat_message(msgs.messages[i*2-1].type).write(msgs.messages[i*2-1].content)


Writing app.py


# Run the streamlit app. Keep this line of codes running and paste the endpoint IP to the url shown in the button followed "npx: installed"

In [4]:
! streamlit run app.py & npx localtunnel --port 8501

[..................] \ rollbackFailedOptional: verb npm-session 4382c83a1f584e5


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.139.77.21:8501

npx: installed 22 in 7.959s
your url is: https://eleven-colts-notice.loca.lt
/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:34: UserWarning: Importing OpenAI from langchain root module is no longer supported. Please use langchain.llms.OpenAI instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:34: UserWarning: Importing ConversationChain from langchain root module is no longer supported. Please use langchain.chains.ConversationChain instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:34: UserWarning: Importing OpenAI from langchain root module is no longer supported. Please use langchain.llms.OpenAI instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/__init__.py: